# Sprint 10 - Projeto Final: Análise Exploratória de Dados e Visualização


In [ ]:
# Importando as bibliotecas

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import io
import re

## 1 - Preparação dos dados

In [ ]:
# Definir o nome do arquivo
file_name = 'rest_data_us_upd.csv'# Arquivo para os próximos passos

print(f"Iniciando Passo 1: Carregamento e Preparação de Dados (Processo Completo)")
print(f"Arquivo de entrada: {file_name}")
print("\n" + "="*50 + "\n")

try:
    # Passo 1: Carregar os dados
    df = pd.read_csv(file_name)
    print(f"Dados carregados. Shape inicial: {df.shape}")
    print("\n" + "="*50 + "\n")

    # Informações de tipos e nulos
    print("--- 1. Informações Iniciais (Tipos e Nulos) ---")
    buffer = io.StringIO()
    df.info(buf=buffer)
    info_output = buffer.getvalue()
    print(info_output)
    print("\n" + "="*50 + "\n")
    
    # Passo 2: Tratar duplicados
    print("--- 2. Tratando Duplicados (nome + endereço) ---")
    initial_rows = df.shape[0]
    key_duplicates_count = df.duplicated(subset=['object_name', 'address']).sum()
    print(f"Duplicados (nome + endereço) encontrados: {key_duplicates_count}")
    
    df = df.drop_duplicates(subset=['object_name', 'address'], keep='first')
    rows_dropped_dupes = initial_rows - df.shape[0]
    
    if rows_dropped_dupes > 0:
        print(f"{rows_dropped_dupes} duplicados removidos.")
    else:
        print("Nenhum duplicado (nome + endereço) encontrado.")
    print(f"Novo shape: {df.shape}")
    print("\n" + "="*50 + "\n")

    # Passo 3: Tratar valores ausentes ('chain')
    print("--- 3. Tratando Valores Ausentes ('chain') ---")
    initial_rows = df.shape[0]
    null_count = df['chain'].isna().sum()
    print(f"Valores ausentes em 'chain' (antes): {null_count}")
    
    if null_count > 0:
        df.dropna(subset=['chain'], inplace=True)
        rows_dropped_na = initial_rows - df.shape[0]
        print(f"{rows_dropped_na} linhas com 'chain' ausente foram removidas.")
    else:
        print("Nenhum valor ausente em 'chain' encontrado.")
    print(f"Novo shape: {df.shape}")
    print("\n" + "="*50 + "\n")

    # Passo 4: Corrigir tipos de dados ('chain' e 'object_type')
    print("--- 4. Corrigindo Tipos de Dados ('chain' e 'object_type') ---")
    
    # Corrigir 'chain' para booleano
    print(f"Tipo de 'chain' (antes): {df['chain'].dtype}")
    # A conversão .astype(bool) é robusta para True/False strings e valores numéricos.
    # Após o dropna, não devemos ter NaNs aqui.
    df['chain'] = df['chain'].astype(bool)
    print("Conversão de 'chain' para booleano concluída.")
    print(f"Tipo de 'chain' (depois): {df['chain'].dtype}")
    
    print("-" * 20) # Separador para clareza
    
    # Corrigir 'object_type' para categoria (NOVA SOLICITAÇÃO)
    print(f"Tipo de 'object_type' (antes): {df['object_type'].dtype}")
    df['object_type'] = df['object_type'].astype('category')
    print("Conversão de 'object_type' para categoria concluída.")
    print(f"Tipo de 'object_type' (depois): {df['object_type'].dtype}")

    print("\n" + "="*50 + "\n")

    # Passo 5: Verificação final (MODIFICADO)
    print("--- 5. Verificação Pós-Limpeza (Final) ---")
    print(f"Shape final: {df.shape}")
    
    print("\n--- Informações Finais (Tipos e Nulos) ---")
    buffer_final = io.StringIO()
    df.info(buf=buffer_final)
    info_output_final = buffer_final.getvalue()
    print(info_output_final)
    
    print("\n--- Primeiras 10 linhas do DataFrame limpo ---")
    print(df.head(10)) # NOVA SOLICITAÇÃO
    
    # Não salvar o arquivo, apenas retornar o DataFrame limpo para uso posterior (SOLICITAÇÃO)
    print("\nDataFrame limpo pronto para uso no restante do projeto.")

except FileNotFoundError:
    print(f"Erro: O arquivo '{file_name}' não foi encontrado.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")


    

## 2 - Análise dos dados

### 2.1 - Proporções dos tipos de estabalecimentos

In [ ]:
print("--- 2.1 Proporções de Tipos de Estabelecimentos ---")
    
# Calculando proporções
type_proportions = (df['object_type'].value_counts(normalize=True) * 100).reset_index()
type_proportions.columns = ['object_type', 'percentage']
    
print("Proporções (%):")
print(type_proportions)
    
# Plotando o gráfico de barras
plt.figure(figsize=(10, 6))
sns.barplot(
    data=type_proportions, 
    x='percentage', 
    y='object_type',
    order=type_proportions['object_type'], # Manter a ordem do value_counts
    palette='muted'
)
plt.title('Proporção de Tipos de Estabelecimento')
plt.xlabel('Porcentagem (%)')
plt.ylabel('Tipo de Estabelecimento')
plt.show()
print("\n" + "="*50 + "\n")

#### Conclusão:  

O gráfico de barras e os dados de proporção mostram que o mercado de LA é vastamente dominado por 'Restaurant' (Restaurante), correspondendo a 75,2% de todos os estabelecimentos.

'Fast Food' (11,1%) e 'Cafe' (4,5%) são os próximos mais comuns, mas com uma participação de mercado significativamente menor. Isso indica que, embora haja concorrência em todos os setores, o seu nicho de 'Café' é muito menos saturado do que o de 'Restaurante'.

### 2.2 - Proporções de estabelecimentos de rede e não

In [ ]:
print("INICIANDO PASSO 2.2")
print("--- 2.2 Proporções de Rede vs. Não Rede ---")
    
# Calculando proporções
chain_proportions = (df['chain'].value_counts(normalize=True) * 100)
    
print("Proporções (%):")
# Mapear True/False para rótulos mais claros na impressão
chain_proportions_labeled = chain_proportions.rename(index={True: 'Rede', False: 'Não Rede'})
print(chain_proportions_labeled)
    
# Plotando o gráfico de pizza
plt.figure(figsize=(7, 7))
    
# Mapear True/False para rótulos
labels = chain_proportions.index.map({True: 'Rede', False: 'Não Rede'})
    
plt.pie(
    chain_proportions, 
    labels=labels, 
    autopct='%1.1f%%', # Formato da porcentagem
    startangle=90,      # Iniciar a 90 graus
    colors=sns.color_palette('muted')[0:2] # Usar cores da paleta 'muted'
)
    
plt.title('Proporção de Estabelecimentos (Rede vs. Não Rede)')
plt.axis('equal') # Assegura que a pizza seja um círculo
plt.legend(labels, loc="best", title="Tipo")
plt.show()
print("\n" + "="*50 + "\n")

#### Conclusão:  

O gráfico de pizza mostra que o mercado de LA é dominado por estabelecimentos independentes. A maioria (61,9%) Não é Rede, enquanto 38,1% são locais que pertencem a uma rede.

### 2.3 -  Tipo de estabelecimento é típico para redes

In [ ]:
print("INICIANDO PASSO 2.3")
print("--- 2.3 Tipo de Estabelecimento Típico para Redes ---")

# Interpretação 1: O tipo mais comum DENTRO das redes (Contagem Absoluta)
print("--- Análise 1: Contagem Absoluta (Qual tipo de rede é mais numeroso?) ---")
# Filtrar apenas os estabelecimentos que são redes
df_chains = df[df['chain'] == True]
    
# Contar os tipos de estabelecimentos dentro das redes
chain_type_counts = df_chains['object_type'].value_counts()
print("Contagem absoluta de locais por tipo (apenas redes):")
print(chain_type_counts)
print(f"\nConclusão (Absoluta): O tipo mais comum DENTRO das redes é '{chain_type_counts.idxmax()}' (N={chain_type_counts.max()}).")
    
print("\n" + "-"*50 + "\n")

# Interpretação 2: O tipo que tem a maior chance (proporção) de SER uma rede
print("--- Análise 2: Proporcional (Qual tipo tem maior chance de ser uma rede?) ---")
    
# Contar o total de cada tipo no dataset geral
total_type_counts = df['object_type'].value_counts()
    
# Calcular a proporção de "rede" dentro de cada tipo
# (Quantos 'Restaurantes' são rede / Total de 'Restaurantes')
# Usamos .reindex para garantir que os totais (denominador) correspondam aos de redes (numerador)
proportion_chain_by_type = (chain_type_counts.reindex(total_type_counts.index, fill_value=0) / total_type_counts * 100).sort_values(ascending=False)
    
print("Porcentagem de estabelecimentos que são 'Rede' (por tipo):")
print(proportion_chain_by_type.round(2))
    
print(f"\nConclusão (Proporcional): O tipo com maior PROPORÇÃO de redes é '{proportion_chain_by_type.idxmax()}' ({proportion_chain_by_type.max():.1f}%).")
    
# Preparar dados para o gráfico
proportion_df = proportion_chain_by_type.reset_index()
proportion_df.columns = ['object_type', 'percentage_is_chain']
    
print("Porcentagem de estabelecimentos que são 'Rede' (dentro de cada tipo):")
print(proportion_df)
    
print("\n" + "-"*50 + "\n")
    
# --- Plotando o gráfico desta proporção ---
print("--- Gerando gráfico da proporção de redes por tipo...")
    
plt.figure(figsize=(10, 6))
sns.barplot(
    data=proportion_df,
    x='percentage_is_chain',
    y='object_type',
    order=proportion_df['object_type'], # Ordem do sort_values
    palette='muted',
    hue='object_type'
)
plt.title('Proporção de Redes Dentro de Cada Tipo de Estabelecimento')
plt.xlabel('Porcentagem que é Rede (%)')
plt.ylabel('Tipo de Estabelecimento')
plt.show()

print("\n" + "="*50 + "\n")

#### Conclusão:  

Os dados mostram duas realidades diferentes:

Dados Absolutos (Volume): Em números absolutos, 'Restaurant' (Restaurante) é o tipo de rede mais comum (2.291 locais).

Dados por Categoria (Propensão): No entanto, a análise proporcional mostra que 'Bakery' Padaria (100%) e 'Cafe' (61,1%) são os modelos de negócio mais propensos a operar como rede.

Resumo: O mercado de 'Restaurante' é vasto, mas dominado por independentes (apenas 31,6% são redes). Já o modelo de 'Café' (seu projeto) é estruturalmente favorável à expansão em rede, o que é um sinal positivo para escalabilidade futura.

### 2.4 - O que caracteriza redes

In [ ]:
print("INICIANDO PASSO 2.4")
print("--- 2.4 Características das Redes (Tamanho vs. Quantidade) ---")

# Filtrar apenas as redes
df_chains = df[df['chain'] == True]

# Agrupar por nome da rede
# Contar o número de locais (count_locations)
# Calcular a média de assentos (mean_seats)
chain_summary = df_chains.groupby('object_name').agg(
    count_locations=('id', 'count'),
    mean_seats=('number', 'mean')
).reset_index()

# Ordenar pelas redes com mais locais
chain_summary = chain_summary.sort_values(by='count_locations', ascending=False)

print("Sumário das Redes (Top 5 em número de locais):")
print(chain_summary.head())

print("\nSumário das Redes (Estatísticas Descritivas):")
# O .describe() responde a pergunta:
print(chain_summary[['count_locations', 'mean_seats']].describe())

# Plotar a distribuição do número de locais por rede
plt.figure(figsize=(10, 6))
sns.histplot(chain_summary['count_locations'], bins=50, kde=False)
plt.title('Distribuição da Quantidade de Locais por Rede')
plt.xlabel('Número de Locais')
plt.ylabel('Contagem de Redes')
plt.xlim(0, 10) # Focar na maioria (75% tem 1 local)
plt.savefig('step_2_4_chain_locations_hist.png')
print("\nGráfico 'step_2_4_chain_locations_hist.png' salvo.")

# Plotar a distribuição da média de assentos
plt.figure(figsize=(10, 6))
sns.histplot(chain_summary['mean_seats'], bins=50, kde=True)
plt.title('Distribuição da Média de Assentos por Rede')
plt.xlabel('Média de Assentos')
plt.ylabel('Contagem de Redes')
plt.xlim(0, 200) # Focar na maioria
plt.show()
print("\n" + "="*50 + "\n")

#### Conclusão:  

As estatísticas descritivas mostram:

 - count_locations (Poucos Estabelecimentos): O mercado de redes é altamente fragmentado. Existem 2.733 redes diferentes. O valor de 75% (terceiro quartil) para count_locations é 1. Isso significa que 75% de todas as redes neste dataset têm apenas 1 local.

 - mean_seats (Pequeno Número de Assentos): A mediana (50%) da média de assentos é 25.5.

As redes são caracterizadas por poucos estabelecimentos (75% têm apenas 1) com um pequeno número de assentos (mediana de 25.5).

Existem exceções (outliers) como "THE COFFEE BEAN & TEA LEAF" (47 locais), mas a regra geral é de redes pequenas.

### 2.5 - Número médio por tipo de estabelecimento

In [ ]:
print("INICIANDO PASSO 2.5")

# --- Média de assentos (Gráfico de Barras) ---
print("--- Média de Assentos por Tipo de Estabelecimento ---")

# Calcular a média de assentos ('number') para cada 'object_type'
avg_seats_by_type = df.groupby('object_type', observed=True)['number'].mean().sort_values(ascending=False).reset_index()

print("Média de assentos por tipo:")
print(avg_seats_by_type)

# Plotar o gráfico de barras
plt.figure(figsize=(10, 6))
ax_bar = sns.barplot(
    data=avg_seats_by_type,
    x='number',
    y='object_type',
    order=avg_seats_by_type['object_type'], # Manter a ordem da média
    palette='muted',
    hue='object_type' # Para evitar o aviso de depreciação
)

plt.title('Média de Assentos por Tipo de Estabelecimento')
plt.xlabel('Média de Assentos')
plt.ylabel('Tipo de Estabelecimento')
plt.savefig('step_2_5a_avg_seats_type_bar.png')
print("\nGráfico 'step_2_5a_avg_seats_type_bar.png' salvo.")

# Responder à pergunta específica
print(f"\nTipo com maior média de assentos: {avg_seats_by_type.iloc[0]['object_type']} (Média: {avg_seats_by_type.iloc[0]['number']:.1f})")

# --- Distribuição de assentos (Boxplot) ---
print("\n" + "-"*50 + "\n")
print("--- Distribuição de Assentos por Tipo (Boxplot) ---")

# Obter a ordem do gráfico anterior para manter consistência
type_order = avg_seats_by_type['object_type'].tolist()

plt.figure(figsize=(12, 7))
ax_box = sns.boxplot(
    data=df,
    x='number',
    y='object_type',
    order=type_order, # Manter a ordem (do maior para o menor avg)
    palette='muted',
    hue='object_type' # Para evitar o aviso de depreciação
)# Remover a legenda duplicada

plt.title('Distribuição de Assentos por Tipo de Estabelecimento')
plt.xlabel('Número de Assentos')
plt.ylabel('Tipo de Estabelecimento')
plt.xlim(0, 250) # Limitar eixo X para focar na maioria (remover outliers extremos)
plt.savefig('step_2_5b_seats_distribution_type_boxplot.png')
print("Gráfico 'step_2_5b_seats_distribution_type_boxplot.png' salvo.")
print("\n" + "="*50 + "\n")

#### Conclusão:

- Gráfico de Barras (Média): O 'Restaurant' (Restaurante) é o tipo com a maior média de assentos (48,1). O seu tipo, 'Cafe' (Café), tem a segunda menor média (25,0), ligeiramente acima de 'Bakery' (Padaria) (21,8).

- Boxplot (Distribuição): Este gráfico mostra que, embora a média de 'Restaurant' seja 48,1, a mediana (a linha central da caixa) é menor (cerca de 30-35). Isso significa que a média é "puxada" para cima por locais muito grandes (os pontos "outliers"). 'Cafe' e 'Bakery' têm distribuições muito mais compactas, confirmando que a maioria desses locais é, de fato, pequena.

Em resumo, 'Restaurant' e 'Fast Food' são categorias com uma grande variação de tamanho, indo de muito pequenos a muito grandes.
Isso contrasta com 'Cafe' e 'Bakery', que são mais consistentes e quase sempre pequenos (possuem poucos ou nenhum outlier).

### 2.6 - Criar coluna nome das ruas

In [ ]:
print("INICIANDO PASSO 2.6")
print("--- 2.6 Extraindo Nomes das Ruas ---")

# Função para extrair a rua usando Regex
def extract_street(address):
    # Estratégia: Tentar encontrar um padrão de endereço comum que termine 
    # com um sufixo de rua (BLVD, ST, AVE, etc.)
    
    # Regex (Plano A): Tenta encontrar o nome + sufixo (ex: N EAGLE ROCK BLVD)
    match = re.search(
        r'((?:\d+\s+)?[A-Z0-9\s]+(?:BLVD|ST|AVE|RD|WAY|DR|LN|PL|CT|CIR))', 
        address, 
        re.IGNORECASE
    )
    
    if match:
        # Se encontrou, pega o grupo 1 (o endereço)
        street_name = match.group(1)
    else:
        # Fallback (Plano B): Se não encontrou sufixo (ex: '100 WORLD WAY 120')
        # Apenas remove suítes/números do final
        street_name = re.sub(r'[,#]\s*.*$', '', address) # Remove , STE 100
        street_name = re.sub(r'\s+\d+$', '', street_name) # Remove ' 120'

    # De qualquer forma, remove o número do prédio do início e formata
    street_name = re.sub(r'^\d+\s+', '', street_name).strip().upper()
    return street_name

# Aplicar a função para criar a nova coluna 'street'
df['street'] = df['address'].apply(extract_street)

print("Extração de ruas concluída.")

# Mostrar o resultado
print("\nExemplo de ruas extraídas (colunas 'address' e 'street'):")
print(df[['address', 'street']].head(10))

print("\n" + "="*50 + "\n")

### 2.7 - Gráfico de dez ruas com o maior número de restaurantes

In [ ]:
print("--- 2.7 Top 10 Ruas por Número de Restaurantes ---")

# 1. Contar os valores na coluna 'street' e pegar as 10 maiores
top_10_streets_counts = df['street'].value_counts().head(10)

# 2. Converter a Series para um DataFrame para plotar
top_10_streets_df = top_10_streets_counts.reset_index()
top_10_streets_df.columns = ['street', 'count']

print("Top 10 Ruas:")
print(top_10_streets_df)

# 3. Plotar o gráfico de barras
plt.figure(figsize=(10, 7))
ax = sns.barplot(
    data=top_10_streets_df,
    x='count',
    y='street',
    order=top_10_streets_df['street'], # Manter a ordem
    palette='muted',
    hue='street' # Para evitar o aviso de depreciação
)

plt.title('Top 10 Ruas por Número de Estabelecimentos')
plt.xlabel('Número de Estabelecimentos')
plt.ylabel('Rua')
plt.show()
print("\n" + "="*50 + "\n")

### 2.8 - Número de ruas que têm apenas um restaurante

In [ ]:
print("INICIANDO PASSO 2.8")
print("--- 2.8 Ruas com Apenas Um Restaurante ---")

# 1. Contar a frequência de cada rua (agora a coluna 'street' existe)
street_counts = df['street'].value_counts()

# 2. Filtrar a contagem para apenas aquelas onde o valor é 1
single_restaurant_streets_count = street_counts[street_counts == 1].count()

# 3. Informações de contexto (Total de ruas)
total_unique_streets = len(street_counts)
percentage = (single_restaurant_streets_count / total_unique_streets) * 100

# Exibir o resultado
print(f"Número de ruas com apenas um restaurante: {single_restaurant_streets_count}")

print(f"\nNúmero total de ruas únicas: {total_unique_streets}")
print(f"Porcentagem de ruas com apenas 1 local: {percentage:.1f}%")
print("\n" + "="*50 + "\n")

#### Conclusão:  

O mercado de LA é muito disperso. 409 ruas têm apenas um restaurante, o que representa 44,0% de todas as ruas únicas no conjunto de dados.


### 2.9 - Distribuição de número de assentos em ruas com muitos restaruantes

In [ ]:
print("INICIANDO PASSO 2.9")
print("--- 2.9 Distribuição de Assentos nas Top 10 Ruas ---")

# 1. Obter a lista das Top 10 ruas (como no Passo 2.7)
top_10_streets_counts = df['street'].value_counts().head(10)
top_10_street_names = top_10_streets_counts.index.tolist()

print("Top 10 ruas sendo analisadas:")
print(top_10_street_names)

# 2. Filtrar o DataFrame original para incluir apenas essas ruas
df_top_streets = df[df['street'].isin(top_10_street_names)]

# 3. Plotar a distribuição de assentos (usando 'number')
plt.figure(figsize=(12, 8))
ax = sns.boxplot(
    data=df_top_streets,
    x='number', # Número de assentos
    y='street', # Rua
    order=top_10_street_names, # Manter a ordem (mais movimentada em cima)
    palette='muted',
    hue='street' # Para evitar o aviso de depreciação
)


plt.title('Distribuição de Assentos nas Top 10 Ruas')
plt.xlabel('Número de Assentos')
plt.ylabel('Rua')
plt.xlim(0, 250) # Limitar o eixo X para melhor visualização (remover outliers extremos)
plt.savefig('step_2_9_seats_top_streets_boxplot.png')

print("\nGráfico 'step_2_9_seats_top_streets_boxplot.png' salvo.")
print("\n" + "="*50 + "\n")

#### Conclusão:

O gráfico boxplot (distribuição de assentos) nas ruas mais movimentadas revela duas tendências principais:

- A "Regra" é Pequena: A maioria dos estabelecimentos nessas ruas concorridas é pequena. A "caixa" (que representa 50% dos locais) e a linha da mediana (o centro da caixa) estão quase sempre abaixo de 50 assentos.

- Muitas Exceções (Outliers): Embora a maioria seja pequena, essas ruas têm uma enorme quantidade de outliers (os pontos individuais à direita). Isso significa que locais muito grandes (100, 150, 200+ assentos) também são muito comuns nessas áreas.

As ruas mais populares de LA abrigam uma mistura de locais: uma alta densidade de locais pequenos (a regra), competindo com muitos locais grandes (as exceções).

# Conclusão Geral:  

**1. Tipo de Restaurante: Café (Apropriado)**

A escolha de abrir um 'Cafe' (Café) é estratégica. A análise (Passo 2.1) mostrou que este é um nicho de mercado (apenas 4,5% do total), o que permite evitar a concorrência direta e saturada dos 'Restaurantes' (que dominam 75% do mercado). Para um projeto inovador e caro como "garçons robôs", focar em um nicho onde a experiência do cliente é mais importante que o volume é o ideal.

**2. Número de Assentos: Pequeno (25-30 assentos)**

A recomendação é um número pequeno de assentos, na faixa de 25 a 30.

A análise (Passo 2.5) mostrou que a média de assentos para 'Cafe' é de 25, com uma distribuição muito consistente e poucos outliers (locais grandes). Isso se alinha perfeitamente com a análise das redes (Passo 2.4), que mostrou que a maioria das redes de sucesso é pequena (mediana de 25,5 assentos). Esta configuração otimiza o custo e foca na eficiência dos robôs, sem tentar competir em volume com os grandes restaurantes.

**3. Possibilidade de Rede: Alta (Com Cautela)**

Sim, o potencial para desenvolver uma rede é alto.

A análise Porcentagem de rede por tipo revelou que 'Cafe' é o segundo tipo de negócio com maior probabilidade de operar como rede (61,1% de todos os cafés pertencem a uma rede). Isso indica que o modelo de 'Café' é estruturalmente adequado para escalabilidade.

Observação: Embora o potencial seja alto, a análise (Passo 2.4) também mostrou que 75% das redes no dataset possuem apenas um local. Isso sugere que a expansão deve ser cautelosa, validando o sucesso da primeira loja-conceito antes de uma expansão agressiva.

# Link Apresentação:  

Presentation: https://1drv.ms/b/c/59a6b343d0bd7f4d/EWB_XSvvCiNEvBVyZog6nj4BtinjuxVHcxwHCAV-3PQoWA?e=bwrodx